In [ ]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
from tqdm import tqdm

In [ ]:
#load data

folder = "/home/anderscm/Documents/Data/SPED"

file_1deg ="/Users/anders/Library/CloudStorage/OneDrive-NTNU/Prosjektoppgave/Data/"\
    "SPED/SPED_640x383x12_20x20_9p26x9p26nm_ScanRot20deg_CL12cm_spot0p5nm_alpha5_"\
    "NBD_10ms_1deg_fixed_scan.zspy"

onedeg = hs.load(file_1deg, lazy=True)


In [ ]:
#Crop around grain f and g

dp = onedeg.inav[290:600,:240]

#set step size calibration

dp.axes_manager[0].scale = 6.92
dp.axes_manager[1].scale = 6.92

In [ ]:
#Define virtual selected area apertures in both grains

ROI_Bright_f = hs.roi.CircleROI(cx=3232.08, cy=463.64, r=79.58, r_inner=0)
ROI_Dark_f = hs.roi.CircleROI(cx=3024.48, cy=463.64, r=79.58, r_inner=0)


ROI_Bright_g = hs.roi.CircleROI(cx=4346.2, cy=581.28, r=100.34, r_inner=0)
ROI_Dark_g = hs.roi.CircleROI(cx=4256.24, cy=802.72, r=100.34, r_inner=0)

In [ ]:
#define function for vsad difference plot

def plot_ROI_diff(ROI_bright, ROI_dark):

    plot =  ROI_bright(dp).nansum()
    bright = ROI_bright(dp).nansum()
    dark = ROI_dark(dp).nansum()

    b = bright.data
    d = dark.data

    plot.data = b-d
    plot.data /= 1250

    plot.plot( axes_off = True, vmin = -1, vmax = 1 , title = "", 
                        cmap = "gray", scalebar_color = "black")


In [ ]:
#VSAD difference for grain g

plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams["axes.unicode_minus"] = False
plot_ROI_diff(ROI_bright=ROI_Bright_g, ROI_dark=ROI_Dark_g)
#plt.savefig("SPED1_VSAD_g.svg")

In [ ]:
#VSAD difference for grain f

plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams["axes.unicode_minus"] = False
plot_ROI_diff(ROI_bright=ROI_Bright_f, ROI_dark=ROI_Dark_f)
#plt.savefig("SPED1_VSAD_f.svg")

In [ ]:
#manually define virtual objective aperture

roi_000 = hs.roi.CircleROI(cx=0.000291292, cy=-0.000194195, r=0.000825327, r_inner=0)
roi_100 = hs.roi.CircleROI(cx=0.00223324, cy=0, r=0.000825327, r_inner=0)
roi_002 = hs.roi.CircleROI(cx=0.000582584, cy=-0.00213614, r=0.000825327, r_inner=0)


#define function for getting objective aperture around peak (h,0,l)

def get_circkle(h,k,l):
    if k != 0 :
        raise ValueError(f"Parameter k need to be 0")
    origin_x, origin_y = 0.000291292, -0.000194195

    one_0_0_vector_x = (0.00990393 - origin_x)/5
    one_0_0_vector_y = (0.00116517 - origin_y)/5
    zero_0_2_vector_x = (0.00165065 - origin_x)/5
    zero_0_2_vector_y = (-0.00922424 - origin_y)/5

    cx = origin_x + one_0_0_vector_x * h + zero_0_2_vector_x*l/2
    cy = origin_y + one_0_0_vector_y * h + zero_0_2_vector_y *l/2 

    return hs.roi.CircleROI(cx = cx, cy = cy,  r=0.000825327, r_inner=0)


In [ ]:
#Calculate full contrast matrix for grain g from all reflection

def get_score_for_g(h,k,l):
    roi_1 = get_circkle(h,0,l)
    pos1 = np.nansum(ROI_Bright_g((roi_1(dp.T)).nansum()).data)
    neg1 = np.nansum(ROI_Dark_g((roi_1(dp.T)).nansum()).data)
    return (pos1-neg1)/neg1

scores = np.zeros((11,13))

for h in tqdm(range(-5,6)):
    for l in range(-6,7):
        scores[h+5,l+6] = get_score_for_g(h,0,l*2)



In [ ]:
#plot the full result

fig,ax = plt.subplots()
img = ax.imshow(scores.T,cmap = "RdBu",origin = "lower", vmin = -1, vmax= 1)
ax.set_xticks(range(11))
ax.set_xticklabels([-5,-4,-3,-2,-1,0,1,2,3,4,5], fontsize =16)
ax.set_yticks(range(13))
ax.set_yticklabels([-12,-10,-8,-6,-4,-2,0,2,4,6,8,10,12], fontsize =16)
cbar = plt.colorbar(img)

cbar.ax.tick_params(labelsize=16)
ax.set_xlabel("h", fontsize =16)
ax.set_ylabel("l", fontsize =16)
plt.tight_layout()
plt.savefig("Full_matrix_grain_g.pdf")

In [ ]:
#Calculate full contrast matrix for grain f from all reflections

def get_score_for_f(h,k,l):
    roi_1 = get_circkle(h,0,l//2)
    pos1 = np.nansum(ROI_Bright_f((roi_1(dp.T)).nansum()).data)
    neg1 = np.nansum(ROI_Dark_f((roi_1(dp.T)).nansum()).data)
    return (pos1-neg1)/neg1

scores_f = np.zeros((11,13))


for h in tqdm(range(-5,6)):
    for l in range(-6,7):
        scores_f[h+5,l+6] = get_score_for_f(h,0,l*2)


In [ ]:
#plot the full result

fig,ax = plt.subplots()
img = ax.imshow(scores_f.T,cmap = "RdBu",origin = "lower", vmin = -1, vmax= 1)
ax.set_xticks(range(11))
ax.set_xticklabels([-5,-4,-3,-2,-1,0,1,2,3,4,5],fontsize =16)
ax.set_yticks(range(13), fontsize =16)
ax.set_yticklabels([-12,-10,-8,-6,-4,-2,0,2,4,6,8,10,12], fontsize = 16)
cbar = plt.colorbar(img)

cbar.ax.tick_params(labelsize=16)
ax.set_xlabel("h", fontsize =16)
ax.set_ylabel("l", fontsize = 16)
plt.tight_layout()
plt.savefig("Full_matrix_grain_f.pdf")

In [ ]:
def compress_matrix(matrix, grain_f = False):
    new_matrix = np.zeros((matrix.shape[0]//2+1, matrix.shape[1]//2+1))

    for i in range(new_matrix.shape[0]):
        for j in range(new_matrix.shape[1]):
            new_matrix[i,j] += matrix[5-i,j+6]
            new_matrix[i,j] -= matrix[5-i,6-j]

            if not grain_f:
                new_matrix[i,j] += matrix[i+5,j+6]
                new_matrix[i,j] -= matrix[i+5,6-j]

                new_matrix[i,j] /= 4
            else:
                new_matrix[i,j] /=2

    return new_matrix.T

In [ ]:
#plot compressed matrix for g
fig,ax = plt.subplots()

img = ax.imshow(compress_matrix(scores), 
                cmap ="RdBu", 
                origin= "lower", 
                vmin = -1, 
                vmax = 1)
plt.colorbar(img)

ax.set_xlabel("h")
ax.set_ylabel("l")

ax.set_xticklabels([0,0,1,2,3,4,5])
ax.set_yticklabels([0,0,2,4,6,8,10,12])

plt.tight_layout()
#plt.savefig("contrast_matrix_grain_g.svg")


In [ ]:
#plot compressed matrix for f
fig,ax = plt.subplots()

img = ax.imshow(compress_matrix(scores_f, grain_f=True), 
                cmap ="RdBu", 
                origin= "lower", 
                vmin = -1, 
                vmax = 1)
plt.colorbar(img)

ax.set_xlabel("h")
ax.set_ylabel("l")

ax.set_xticklabels([0,0,1,2,3,4,5])
ax.set_yticklabels([0,0,2,4,6,8,10,12])

plt.tight_layout()
plt.savefig("contrast_matrix_grain_f.svg")
